In [18]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 11 11:02:03 2021

@author: Christian
"""

from mip import Model, xsum, maximize, BINARY

from Co2_price import df_data
from electricity_demand import electricity_demand
from Electricity_Price0 import Electricity_price_pivot0
from Electricity_Price1 import Electricity_price_pivot1
from gas_price import gas_p
from heat_demand import heat_demand_pivot
from heat_price import heat_p
from elec_effciency import elec_eff
from electricity_capacity import elec_capacity
from heat_capacity import heat_capacity
from heat_boiler_capacity import heat_cap
from heat_boiler_efficiency1 import heat_eff



elec_demand = electricity_demand.values.tolist()
elec_price_wholesale = Electricity_price_pivot0
elec_price_household_32_05 = Electricity_price_pivot1
co2_price = df_data
gas_price = gas_p
heat_demand_1 = heat_demand_pivot
heat_price_1 = heat_p


el_price = elec_price_wholesale.values.tolist() # Electricity price
del_t = .25 # duration of time step
el_demand = electricity_demand.values.tolist() # Electicity demand


gas_pp = gas_price["Preis"].values.tolist() # Gas price for different plants
em_fc = 0.2 # emission factor
co2_p = co2_price["mean_CO2_tax"].values.tolist() # CO2 Price
capacity_el = elec_capacity.values.tolist() # maximum capacity of electricity for the power plants
capacity_ht = heat_capacity.values.tolist() # maximum capacity of heat for the power plants
heat_demand = heat_demand_1.values.tolist() # heat demand
heat_price = heat_price_1.values.tolist() # heat price
heat_ratio = 400/385 # heat ratio
eff_plants = elec_eff.values.tolist() # the efficiency of the power plants


capacity_ht_boiler = heat_cap.values.tolist()
eff_boiler = heat_eff.values.tolist()




T = range(len(el_price))
I = range(len(gas_pp))

m = Model("Maximizing profit", sense=maximize)
 

#variable
y_t = [[m.add_var(lb = 0) for i in I] for t in T] # Electricity generation
el_sold = [[m.add_var(lb = 0) for i in I] for t in T] # electricity sold 
# el_sold = [m.add_var(lb = 0) for t in T] # electricity sold 
x_t = [[m.add_var(lb = 0) for i in I] for t in T] # Fuel consumption
z_t = [[m.add_var(lb = 0) for i in I] for t in T] # Heat generation
# objective function

# Revenue = Electricity price * demand/generation
# Cost = Fuel price = gas price + emission_factor * co2 price

# Max Proft = Revenue - Cost



In [23]:
co2_p

[17.43,
 17.43,
 17.43,
 17.43,
 19.93,
 21.32,
 28.0,
 28.045,
 28.09,
 26.71,
 27.884999999999998,
 29.06,
 24.34,
 24.05,
 24.72,
 25.06,
 25.67,
 25.573333333333334,
 25.476666666666667,
 25.38,
 25.12,
 25.21,
 24.89,
 24.57,
 24.636666666666667,
 24.703333333333333,
 24.77,
 24.81,
 24.16,
 23.9,
 24.05,
 24.55,
 24.125,
 23.7,
 18.41,
 20.41,
 22.41,
 27.66,
 28.115000000000002,
 28.57,
 27.23,
 23.83,
 29.74,
 24.58,
 24.47,
 24.723333333333333,
 24.976666666666667,
 25.23,
 25.32,
 25.89,
 25.82,
 25.79,
 25.446666666666665,
 25.103333333333335,
 24.76,
 24.36,
 24.44,
 23.78,
 23.79,
 24.494999999999997,
 25.2,
 23.39,
 23.54,
 18.32,
 20.37,
 22.42,
 28.2,
 26.47,
 29.07,
 26.810000000000002,
 24.55,
 29.18,
 22.13,
 21.316666666666666,
 20.503333333333334,
 19.69,
 18.68,
 15.69,
 16.76,
 16.43,
 16.246666666666666,
 16.063333333333333,
 15.88,
 17.1,
 17.85,
 17.69,
 16.73,
 16.953333333333333,
 17.176666666666666,
 17.4,
 18.02,
 20.775,
 23.53,
 24.01,
 21.86,
 19.71,
 2

In [24]:
# m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)


In [17]:
m.objective = xsum((el_price[t][i] * del_t * el_sold[t][i]) - (x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t][i] * heat_price[t][i]) for t in T for i in I)


KeyError: 0

In [2]:
m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)

# constraints

for t in T:
    # m += xsum(y_t[t][i] for i in I) == el_demand[t]
    for j in J: 
        m += z_t[t][j] <= capacity_ht_boiler[j] # heat generation <= maximum capacity of heat of the plant 
        m += x_t[t][j] == z_t[t][j]/eff_boiler[j] # fuel consumption = Heat generation / efficiency of the plants
    
    for k in K:
        m += y_t[t][k] <= capacity_el[k] # electricity generation <= maximum capacity of electricity of the plant
        m += z_t[t][k] <= capacity_ht[k] # heat generation <= maximum capacity of heat of the plant 
        
        m += x_t[t][k] == y_t[t][i]/eff_plants[k] # fuel consumption = Electricity generation / efficiency of the plants 
        
        m += y_t[t][k] == heat_ratio * z_t[t][k] # electricity generation >= heat ratio * heat generation



    for i in I:
        m += heat_demand[t] <= z_t[t][i] # heat demand >= Heat generation  
        m += y_t[t][i] == el_sold[t] + el_demand[t]





status = m.optimize()
obj= m.objective_value

41.709999999999994

In [3]:
status

<OptimizationStatus.OPTIMAL: 0>

In [7]:
r = [3, 5, 8, 3, 6, 5, 7, 3]


In [8]:
type(r)

list